# Patient Pulse AI - Building the RAG Engine

## Install Libraries

In [4]:
#!pip install langsmith==0.3.18 --force-reinstall
#!pip install langgraph
#!pip install pipdeptree 
#!pip install tiktoken
#!pip install --upgrade langchain langchain-text-splitters
#!pip install llama-index
#!pip install --upgrade llama-index langchain
#!pip install llama-index-embeddings-langchain
#!pip install llama-index-llms-langchain
#!pip install -U langchain-google-vertexai
#!pip install -U google-cloud-aiplatform
#!pip install -U langchain-google-vertexai
#!pip install llama-index-vector-stores-chroma
#!pip install llama-index-embeddings-langchain
#!pip install langchain-community
#!pip install openpyxl
#!pip install gradio

### Import the Libraries

In [5]:
import getpass
import os
import base64
import pandas as pd
import json
import vertexai
import tiktoken # A popular tokenizer, though not directly from google-generativeai
import time
from vertexai.generative_models import GenerativeModel, Part, SafetySetting
import warnings
warnings.filterwarnings("ignore")
from langchain_text_splitters import RecursiveCharacterTextSplitter

project_name = !(gcloud config get-value core/project)
project_name = project_name[0]
from llama_index.llms.langchain import LangChainLLM
from langchain_google_vertexai import VertexAIEmbeddings


from llama_index.core.schema import Document
# import
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
#from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from IPython.display import Markdown, display
import chromadb
from llama_index.core import Settings

In [2]:
#!pip install langsmith==0.3.18 --force-reinstall
#!pip install langgraph

In [3]:
#!pip install pipdeptree 

### Initializing the Models

#### Embedding Model

In [6]:

embeddings = VertexAIEmbeddings(model_name="text-embedding-005")

text = "This is a test document."

query_result = embeddings.embed_query(text)

#### Gemini Model

In [5]:
#Gemini
class LLM_model:
    def __init__(self, project, location, model_name):
        self.project = project
        self.location = location
        self.model_name = model_name
        self.generation_config = {
            "max_output_tokens": 5000,
            "temperature": 1,
            "top_p": 0.8,
        }
        self.safety_settings = [
            SafetySetting(
                category=SafetySetting.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
                threshold=SafetySetting.HarmBlockThreshold.OFF
            ),
            SafetySetting(
                category=SafetySetting.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
                threshold=SafetySetting.HarmBlockThreshold.OFF
            ),
            SafetySetting(
                category=SafetySetting.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
                threshold=SafetySetting.HarmBlockThreshold.OFF
            ),
            SafetySetting(
                category=SafetySetting.HarmCategory.HARM_CATEGORY_HARASSMENT,
                threshold=SafetySetting.HarmBlockThreshold.OFF
            ),
        ]
        vertexai.init(project=self.project, location=self.location)
        self.model = GenerativeModel(self.model_name)

    def complete(self, text):
        chat = self.model.start_chat()
        result=chat.send_message([text],
                                 generation_config=self.generation_config,
                                 safety_settings=self.safety_settings
        )
        text_result=result.candidates[0].content.parts[0].text
        return text_result
# Example usage:
llm_gemini = LLM_model(project_name, 
                       "us-central1", 
                       "gemini-2.5-pro")

### Loading the text reviews data

In [7]:
documents = SimpleDirectoryReader("/home/jupyter/reviews/").load_data()

In [8]:
# create client and a new collection
chroma_client = chromadb.EphemeralClient()
chroma_collection = chroma_client.create_collection("quickstart")

In [10]:
#!pip install llama-index-embeddings-langchain
#!pip install llama-index-llms-langchain

In [9]:

from llama_index.core.settings import Settings
#llm_gemini

In [10]:
from langchain_google_vertexai import ChatVertexAI

llm_gemini = ChatVertexAI(model_name="gemini-2.5-pro", temperature=0)

### Chuking and Indexing 

In [11]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)


Settings.llm = llm_gemini
Settings.chunk_size = 500
Settings.chunk_overlap = 200
Settings.embed_model = embeddings
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embeddings
)

In [14]:

query_input = """ I want to go for my cholestrol checks, which hospital expertise is best?"""

### Examining as Retriever

In [15]:
retriever = index.as_retriever(similarity_top_k=5, retriever_mode="llm")
nodes = retriever.retrieve(query_input)
print(len(nodes))
#print(nodes)
print(type(nodes))

5
<class 'list'>


#### Examining the Top 5 Retrievals

In [16]:
nodes_dicts = {}
for i in range(len(nodes)):
    nodes_dicts[f'nodes_dict_{i}'] = nodes[i].to_dict()
    print(nodes_dicts[f'nodes_dict_{i}'].keys())
    print(nodes_dicts[f'nodes_dict_{i}']['score'])


from IPython.display import HTML
for key, value in nodes_dicts.items():
    node = value['node']
    text = node.get('text', '')
    file_path = node['metadata'].get('file_path', '')
    file_name = node['metadata'].get('file_name', '')
    score = value.get('score', 0)
    #print(score)
    display(HTML('<span style="color:green; font-size:20px;">START OF RETRIEVEAL TEXT - ********************************.</span>'))
    display(HTML(f'<span style="color:blue; font-size:15px;">Similarty Score: {score}</span>'))
    display(HTML(f'<span style="color:blue; font-size:15px;">File Name: {file_name}</span>'))
    print(text)
    display(HTML('<span style="color:red; font-size:20px;">END OF RETRIEVEAL TEXT - ********************************.</span>'))

dict_keys(['node', 'score', 'class_name'])
0.468560514467231
dict_keys(['node', 'score', 'class_name'])
0.4677321376760027
dict_keys(['node', 'score', 'class_name'])
0.46397262200348877
dict_keys(['node', 'score', 'class_name'])
0.4591092594937437
dict_keys(['node', 'score', 'class_name'])
0.45544940161411995


--- End of Review ---

--- Review #20 ---
Datetime: 04/08/2025 14:25:33
User: Dr. Martha Tara Lee
Review: My experience at Gleneagles Hospital was marked by outstanding care, professionalism, and even a bit of humor, which made all the difference during my recovery. From the initial consultation to post-operative follow-ups, the staff’s dedication to patient well-being was evident. The state-of-the-art facilities and collaborative approach among the medical team provided a sense of security and trust throughout my treatment.

I would like to especially thank the nurses in Ward 6 for their exceptional care. Not only were they incredibly attentive and kind, but their light-hearted humor helped me stay positive during a challenging time. Their jokes and playful banter made the atmosphere feel more relaxed, turning an otherwise stressful experience into one where I truly felt at ease.

Additionally, I want to express my appreciation for the administrative and medical staff at Gleneagles wh

If your insurance covers other health screening providers, avoid Gleneagles at all cost.
--- End of Review ---

--- Review #139 ---
Datetime: 03/28/2023 15:25:07
User: Min-Ren Seah
Review: many specialists here
--- End of Review ---

--- Review #140 ---
Datetime: 03/27/2023 23:07:43
User: t
Review: not recommended. Doctor was not professional at all, even though he is quite middle aged and expected to be more experienced. The staffs that assisted with the measuring of height and weight, blood pressure were nice and friendly (thus the 2 stars given). Would def not recommend this hospital in general and suggest to go to other hospitals for a better experience.
--- End of Review ---

--- Review #141 ---
Datetime: 03/27/2023 08:34:39
User: J P
Review: Been waiting since 10.30am , post X-ray, for a doctors stamp & note on one of my documents. It’s now 4pm.
First I was told on the phone it can be done within the day and I clarified and theysaid it will be couple of hours so I came early. Was

It’s supposedly chef-designed meals but poorer quality & value-for-money than before. Pls bring back the old food court (miss the ban mian, chicken rice, nasi padang at reasonable prices of $5 to $8 per meal). I’m sure the staff and patients will appreciate that!
--- End of Review ---

--- Review #136 ---
Datetime: 04/02/2023 13:02:58
User: Ai Tee Cheryl Ng
Review: My child just have a day op at Gleneagles hospital and I’m impressed by the nurses. They are very caring and helpful throughout our stay and make sure that my child is not traumatised at the end of the day.
--- End of Review ---

--- Review #137 ---
Datetime: 03/30/2023 06:20:35
User: YL Tan
Review: Not impressed by the service. Had a day surgery and the nurses had different versions of whether should I take any painkillers or not when my doctor already informed them. Nurses should learn to be proactive and give specific instructions to patients who are in great pain and help them through rather than just standing there and 

Relatively easy to get to but make sure your bills can be covered by private insurance or Medicare instead of OOP (out-of-pockrt).
--- End of Review ---

--- Review #412 ---
Datetime: 01/14/2016 01:32:23
User: Tan Seow Ching
Review: nan
--- End of Review ---

--- Review #413 ---
Datetime: 01/12/2016 14:55:39
User: Tareq Salahuddin
Review: Living donor liver transplant is performed in this hospital at Asian Centre for Liver Translaplant.
--- End of Review ---

--- Review #414 ---
Datetime: 12/18/2015 09:11:05
User: Karuna Kwan
Review: Pelayanan yang baik sayangnya harus bayar harga.
--- End of Review ---

--- Review #415 ---
Datetime: 12/01/2015 08:42:57
User: Sameer Alfayez
Review: Sincerity and professional and ethical dealing with patients
--- End of Review ---

--- Review #416 ---
Datetime: 11/24/2015 02:18:11
User: Adrian Lim
Review: Here for a health screening. Pros - food and drinks provided. Cons - Long waiting times for each test. At least 2 hours wait for all to be completed.


We reached the lab at 8.15 and we hv to wait almost an hour to take the blood test . Why is it so??? We hv to wait for almost an hour . This has never happen before since we hv been patrons at the hospital for almost 10 years . I think image enhancement is good for the hospital but service level has to be raised up.
--- End of Review ---

--- Review #23 ---
Datetime: 02/20/2025 10:18:43
User: Joanna Chua
Review: I must review this person who took a blood test for my mother-in-law at Gleneagle Hospital. Sivasangari Phlebotomist is really good in taking blood for blood test. No trace of pain and blue black on my 95 mother-in-law arm. Must praise her on her experiences and on the job she's doing.
--- End of Review ---

--- Review #24 ---
Datetime: 02/18/2025 00:11:47
User: Szeting Xu
Review: nan
--- End of Review ---

--- Review #25 ---
Datetime: 01/26/2025 17:25:39
User: Connor Fenwick
Review: Usually every year go for a checkup and have been clean smooth and perfect, aswell as the stuff

In [17]:
query_input = """ I want to go for my cholestrol checks, which hospital expertise is best?"""

In [18]:
query_engine = index.as_query_engine()
response = query_engine.query(query_input)
display(Markdown(f"<b>{response}</b>"))

<b>Based on the information provided, one user who visits Gleneagles every six months for a heart checkup, which includes blood and urine tests at the Parkway lab centre, expressed disappointment with the service level. This user found the reception at the lab counter to be "very very slow" and recommended avoiding Gleneagles for health screenings if your insurance covers other providers.

Other reviews mention long waiting times, with one person waiting from 10:30 am to 4:00 pm for a doctor's note post-X-ray, and another describing a doctor as unprofessional.

However, another patient had an outstanding experience, highlighting the professionalism, state-of-the-art facilities, and exceptional care from the nurses in Ward 6. This individual also noted that the hospital has many specialists.</b>

In [19]:
from llama_index.core import load_index_from_storage
from llama_index.core import StorageContext

# rebuild storage context
#storage_context = StorageContext.from_defaults(persist_dir="index")
doc_summary_index = load_index_from_storage(storage_context)

In [20]:
query_engine = doc_summary_index.as_query_engine(
    response_mode="tree_summarize", use_async=True
)

In [21]:
query_engine

### Moving to Documment Summary Based Indexing

In [12]:
from llama_index.core import SimpleDirectoryReader, get_response_synthesizer
from llama_index.core import DocumentSummaryIndex
from llama_index.llms.openai import OpenAI
from llama_index.core.node_parser import SentenceSplitter

In [13]:
splitter = SentenceSplitter(chunk_size=700)

In [14]:
# default mode of building the index
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize", use_async=True
)
doc_summary_index = DocumentSummaryIndex.from_documents(
    documents,
    llm=llm_gemini,
    transformations=[splitter],
    response_synthesizer=response_synthesizer,
    show_progress=True,
)

Parsing nodes:   0%|          | 0/8 [00:00<?, ?it/s]

Summarizing documents:   0%|          | 0/8 [00:00<?, ?it/s]

current doc id: 16ca3b9f-0eba-4ed5-9cd1-c5418d2b796a
current doc id: 41ef03bb-3ceb-4168-91dd-b342dc8a3fb3
current doc id: be222ace-d333-4b43-ad42-028345219637
current doc id: 8cfaddfb-7824-402e-9b85-f98792d43304
current doc id: 261efe86-5ade-4587-bce4-479c1e01f289
current doc id: 87927d30-0bfa-474e-97e9-1a10138171d1
current doc id: a47938b8-f1ca-4fbc-b8dc-b403962eb6f6
current doc id: 38cfbe78-15c5-4794-8db4-fa79efeff3ad


Generating embeddings:   0%|          | 0/8 [00:00<?, ?it/s]

In [15]:
query_engine = doc_summary_index.as_query_engine(
    response_mode="tree_summarize", use_async=True
)

In [31]:
from IPython.display import display, HTML, Markdown

In [32]:
query_input = """

what are the recent issues reported for KK Women hospital?


"""

In [33]:

def show_banner(text, color="black", size=15):
    try:
        from IPython.display import display, HTML
        display(HTML(f'<span style="color:{color}; font-size:{size}px;">{text}</span>'))
    except Exception:
        print(text)

def show_markdown(text, size=18):
    try:
        from IPython.display import display, Markdown
        display(Markdown(f"<span style='font-size:{size}px;'>{text}</span>"))
    except Exception:
        print(text)


In [40]:
query_input = """Which reviews mention positive experiences with specific doctors or nurses, and what qualities were praised?"""

In [1]:
#response = query_engine.query(query_input)

In [ ]:

show_banner("LLM Retrieved answer ********************************.", color="green")
show_markdown(f"**{response}**", size=20)
show_banner("END OF RETRIEVAL TEXT - ********************************.", color="red")


In [ ]:
display(HTML('<span style="color:green; font-size:15px;"> LLM Retrieved answer ********************************.</span>'))

display(Markdown(f"<span style='font-size: 20px;'>**{response}**</span>"))

display(HTML('<span style="color:red; font-size:15px;">END OF RETRIEVEAL TEXT - ********************************.</span>'))

In [ ]:
response = query_engine.query(query_input)

In [ ]:
Which reviews mention positive experiences with specific doctors or nurses, and what qualities were praised?


### 🧪 **RAG Engine Test Questions**

1. **What complaints were raised about the Pelvic Floor Disorder Centre at SGH, and how was the hospital's response perceived?**

2. **Summarize the experience of Priscillia Tan at the Head & Neck Centre, including wait times and staff behavior.**

3. **Which reviews mention positive experiences with specific doctors or nurses, and what qualities were praised?**

4. **What are the most common issues patients faced regarding appointment scheduling and wait times at SGH?**

5. **Compare the experiences of patients at SGH versus other hospitals like NUH or Alexandra Hospital.**

6. **Identify any reviews that discuss financial concerns or billing practices at SGH. What sentiments are expressed?**

7. **What feedback was given about SGH’s emergency department (A&E), and how did patients describe their treatment there?**

8. **Are there any reviews that mention SGH’s food quality or hospital facilities? What were the impressions?**

9. **Which reviews reflect concerns about empathy and professionalism among SGH staff, and what examples are given?**

10. **What suggestions or requests did patients make for improving SGH’s services or systems?**



### Simple Search Engine is now Ready

In [44]:
import time

In [ ]:
while True:
    query_input = input("User: ")
    start_time = time.time()
    response = query_engine.query(query_input)
    print("User query****************************************************:",query_input)
    #print("LLM query#######################:",output_text)
    if query_input == "exit":
        break
    #response = Agent_chat(output_text)
    display(HTML('<span style="color:green; font-size:15px;"> LLM Retrieved answer ********************************.</span>'))

    display(Markdown(f"<span style='font-size: 20px;'>**{response}**</span>"))

    display(HTML('<span style="color:red; font-size:15px;">END OF RETRIEVEAL TEXT - ********************************.</span>'))
    #print(f"Agent: {output_text}")
    print("#############################################################")
    elapsed_time = time.time() - start_time
    print(f" Answered in {elapsed_time:.2f} seconds.\n")
    #chat_engine.reset()

User:  •	How do patients compare the experience of giving birth at KKH versus Mount Alvernia?


User query****************************************************: •	How do patients compare the experience of giving birth at KKH versus Mount Alvernia?


<span style='font-size: 20px;'>**Based on patient accounts, experiences at Mount Alvernia are viewed unfavorably when compared to KKH.

One patient who gave birth at both hospitals described the experience at Mount Alvernia as "one of my worst" and "totally incomparable" to the private ward at KKH. This individual found the nurses and lactation consultants at KKH's private ward to be helpful, experienced, and attentive. In contrast, at Mount Alvernia, they felt the nurses were unhelpful and dismissive of their pain.

Another patient described his wife's stay at Mount Alvernia as "torturing" due to "rude & sacarastic" nurses. He stated that Mount Alvernia's "knowledge and service standard is far off" from KKH's, noting there is "plenty of room for improvement."

Other comments include:
*   A patient who was dissatisfied with the pediatric care their child received at Mount Alvernia expressed that they "should have gone to kk."
*   A reviewer noted that government hospitals like KKH provide better services than the A&E at Mount Alvernia.

Patient experiences at Mount Alvernia are varied:
*   **Positive experiences** include descriptions of the hospital as "nothing short of perfect" and the decision to deliver there as the "best decision ever." Nurses have been praised as "impeccable," "attentive," "sweet," "patient," and "caring."
*   **Negative experiences** include reports of "traumatic and abusive" births, rude and impatient nurses, and rough handling of a newborn. One patient's wife was subjected to nurses grabbing her breasts and pinching her nipples without warning. Other issues cited are poor service standards, unhelpful staff regarding breastfeeding issues, billing problems, and a lack of cleanliness.**</span>

#############################################################
 Answered in 252.54 seconds.



User:  •	Which hospital in Singapore is most frequently praised for short waiting times in A&E?


...

User:  •	What was the average waiting time for patients at KKH Children’s Emergency in the past 6 months?


User query****************************************************: •	What was the average waiting time for patients at KKH Children’s Emergency in the past 6 months?


<span style='font-size: 20px;'>**Based on the information provided, a specific average waiting time for patients at the KKH Children’s Emergency in the past six months is not available.

However, individual experiences indicate a wide range of wait times. Some patients have reported extensive waits of 3, 4, 5, 6, 7, and even up to 12 hours to see a doctor. One nurse reportedly told a patient that a "3-hours waiting is 'the normal waiting time'." Other long waits mentioned include waiting for over 2.5 hours, 86 minutes, 90 minutes, and one instance where an estimated 190-minute wait turned into over 260 minutes. A shortage of doctors was given as a reason for a long wait on one occasion.

Conversely, some visitors have had faster experiences, describing the process as "fast and smooth" or being seen in under two hours from arrival to medication collection. Waits of around one to two hours have also been reported. In one case, a child with an open wound was attended to first despite a crowd.

It has also been noted that the estimated waiting times displayed on screens can be unreliable and may increase significantly during the wait.**</span>

#############################################################
 Answered in 476.87 seconds.



User:  exit


### Simple RAG search Engine with Gradio UI

In [17]:
import gradio as gr

def gradio_interface(query_input):
    if query_input == "exit":
        return "Exiting..>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>.", ""
    start_time = time.time()
    response = query_engine.query(query_input)
    elapsed_time = time.time() - start_time
    print(f" Answered in {elapsed_time:.2f} seconds.\n")
    #reset_chat()
    return response

iface = gr.Interface(
    fn=gradio_interface,
    inputs=gr.Textbox(lines=2, placeholder="Enter your query here..."),
    title="Patient Pulse AI",
    outputs=[gr.Textbox(label="Chat Response",lines=10)],
    description="Patient Pulse AI, Question Answering Engine on hospital Google reviews",
    live=False
)

iface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://65195fd78360e28e5c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


 Answered in 492.18 seconds.

